In [ ]:
 !pip install pytorch_tabular -q

In [ ]:
!pip install --upgrade --force-reinstall pandas

In [ ]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig

In [ ]:
import pandas as pd
import numpy as np
import sklearn
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [ ]:
features = [c for c in train.columns if c not in ("target","row_id")]
target="target"
x_test=test[features]
y = train[target]
X = train[features]
numerical_features = X.columns

In [ ]:
num_col_names= numeric_feature_names=list(numerical_features)
cat_col_names =[]

In [ ]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names)

In [ ]:
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=1000,
    gpus=1, #index of the GPU to use. 0, means CPU
)

In [ ]:
optimizer_config = OptimizerConfig()

In [ ]:
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-256-128-64",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3
)

In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [ ]:
tabular_model.fit(train=train)

In [ ]:
test_preds = tabular_model.predict(x_test)

In [ ]:
test_preds

In [ ]:
test_preds['prediction']

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
sub['target'] = test_preds['prediction']

In [ ]:
sub.groupby(['target']).count()

In [ ]:
sub.to_csv('submission.csv', index=False)